### Zadanie z danymi oznakowanymi

Po co stosować metodę uczenia nienadzorowanego do oznakowanych danych? 

Z czystej ciekawości, żeby sprawdzić jak sobie ta metoda poradzi.

Jednym z bardziej popularnych treningowych zbiorów danych jest zbiór danych o [winach](https://www.google.pl/search?q=kaggle+wine+dataset).

Weźmy przykładowy.

In [ ]:
import pandas as pd

wine_dataset = pd.read_csv('data/wine.csv')
wine_dataset

In [ ]:
wine_dataset['Class']

In [ ]:
wine_dataset['Class'].unique()

In [ ]:
y = wine_dataset.pop('Class')
X = wine_dataset

In [ ]:
y

In [ ]:
X

### Model

In [ ]:
from sklearn.cluster import KMeans

model = KMeans(n_clusters=3)  # n_init - check docs
model.fit(X)

In [ ]:
y_kmeans = model.predict(X)
print(y_kmeans[:10])

Porównajmy teraz przypisania (y_kmeans) z etykietami (y)

In [ ]:
df = pd.DataFrame({'labels': y, 'clusters': y_kmeans})
df

In [ ]:
ct = pd.crosstab(df['labels'], df['clusters'])
ct

Wyniki kiepskie. Jednym z możliwych powodów jest brak standaryzacji danych. Być może dane mają bardzo różną wariancję, co przeważnie bardzo przeszkadza w poprawnym grupowaniu. Sprawdźmy to.

In [ ]:
X.describe()

Wyświetlmy przykładowe kolumny

In [ ]:
X.columns

In [ ]:
import plotly.express as px

fig = px.scatter(X[['OD280/OD315 of diluted wines', 'Proline']], 'OD280/OD315 of diluted wines', 'Proline')
fig.show()

Niby ok, ale dopiero jak ujednolicimy skalę na obu osiach, to będziemy w stanie porównać rozkłady obu zmiennych.

In [ ]:
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)
fig.show()

Teraz wyraźnie widać potencjalny problem. Przy tak zróżnicowanych skalach trudno opierać algorytm o pojęcie odległości pomiędzy punktami.
Wynik będzie całkowicie zdeterminowany przez zmienną o największej wariancji. Pozostałe zmienne będą miały znikomy wpływ na wynik.

Ustandaryzujmy dane.

In [ ]:
from sklearn.preprocessing import StandardScaler

# tworzymy obiekt skalujący
scaler = StandardScaler()

# skalujemy dane
scaler.fit(X)
data_scaled = scaler.transform(X)

I teraz spójrzmy jeszcze raz na dane.

In [ ]:
X_scaled = pd.DataFrame(data_scaled, columns=X.columns)
X_scaled.describe()

In [ ]:
fig = px.scatter(X_scaled[['OD280/OD315 of diluted wines', 'Proline']], 'OD280/OD315 of diluted wines', 'Proline')
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)
fig.show()

I teraz powtórzmy całość dla ustandaryzowanych danych.

In [ ]:
model = KMeans(n_clusters=3)  # n_init - check docs
model.fit(X_scaled)

In [ ]:
y_kmeans = model.predict(X_scaled)
print(y_kmeans[:10])

In [ ]:
df = pd.DataFrame({'labels': y, 'clusters': y_kmeans})
df

Wygląda obiecująco

In [ ]:
ct = pd.crosstab(df['labels'], df['clusters'])
ct

Udało się z dużą dokładnością odnaleźć właściwą segmentacje danych.

Wyniku niestety, nie potrafimy zwizualizować, ponieważ pracujemy w 13-wymiarowej przestrzeni.